In [3]:
# MySQL 연결
import pymysql
from sqlalchemy import create_engine  # 연결 관리용으로만 가볍게

# 데이터 처리/시각화
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# 외부 API 호출하기
import requests

# .env 파일 활용하기
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [6]:
conn = pymysql.connect(
    host=os.getenv('MYSQL_HOST'),
    port=int(os.getenv('MYSQL_PORT')),
    user=os.getenv('MYSQL_USER'),
    password=os.getenv('MYSQL_PASSWORD'),
    database=os.getenv('MYSQL_DATABASE'),
    charset='utf8mb4'
)

cursor = conn.cursor()

# 연결 확인
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"MySQL 버전: {version[0]}")

MySQL 버전: 8.0.33


In [8]:
# db_connector.py 파일로 저장해서 재사용
class MySQLConnector:
    def __init__(self):
        self.conn = None
        self.cursor = None
        
    def connect(self):
        """MySQL 연결"""
        self.conn = pymysql.connect(
            host=os.getenv('MYSQL_HOST'),
            port=int(os.getenv('MYSQL_PORT')),
            user=os.getenv('MYSQL_USER'),
            password=os.getenv('MYSQL_PASSWORD'),
            database=os.getenv('MYSQL_DATABASE'),
            charset='utf8mb4'
        )
        self.cursor = self.conn.cursor(pymysql.cursors.DictCursor)
        print("✅ MySQL 연결 성공")
        return self
    
    def execute_query(self, query, params=None):
        """쿼리 실행"""
        self.cursor.execute(query, params)
        self.conn.commit()
        return self.cursor
    
    def fetch_df(self, query, params=None):
        """결과를 DataFrame으로 반환"""
        return pd.read_sql(query, self.conn, params=params)
    
    def close(self):
        """연결 종료"""
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print("✅ MySQL 연결 종료")
    
    def __enter__(self):
        return self.connect()
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

In [10]:
create_regions_table = """
CREATE TABLE IF NOT EXISTS regions (
    region_code VARCHAR(10) PRIMARY KEY COMMENT '지역코드',
    region_name VARCHAR(50) NOT NULL COMMENT '지역명',
    parent_code VARCHAR(10) COMMENT '상위 지역코드',
    region_level TINYINT NOT NULL COMMENT '지역 레벨 (1:시도, 2:시군구)',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    INDEX idx_parent_code (parent_code),
    INDEX idx_region_level (region_level)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='지역 정보';
"""

cursor.execute(create_regions_table)

0

In [16]:
data = pd.read_csv('법정동코드 전체자료.txt'
                  ,sep='\t'
                  ,encoding='EUC-KR')

In [41]:
data.head

<bound method NDFrame.head of             법정동코드                  법정동명 폐지여부
0      1100000000                 서울특별시   존재
1      1111000000             서울특별시 종로구   존재
2      1111010100         서울특별시 종로구 청운동   존재
3      1111010200         서울특별시 종로구 신교동   존재
4      1111010300         서울특별시 종로구 궁정동   존재
...           ...                   ...  ...
49856  5280042024    전북특별자치도 부안군 위도면 대리   존재
49857  5280042025   전북특별자치도 부안군 위도면 거륜리   존재
49858  5280042026   전북특별자치도 부안군 위도면 식도리   존재
49859  5280042027  전북특별자치도 부안군 위도면 상왕등리   존재
49860  5280042028  전북특별자치도 부안군 위도면 하왕등리   존재

[49861 rows x 3 columns]>

In [65]:
폐지_지역 = data[data['폐지여부']!='존재']
존재_지역 = data[data['폐지여부']=='존재'].copy()

In [66]:
존재_지역['공백개수'] = 존재_지역['법정동명'].str.count(' ')

print(존재_지역.tail(100))

            법정동코드                  법정동명 폐지여부  공백개수
49761  5280025032   전북특별자치도 부안군 부안읍 신흥리   존재     3
49762  5280031000       전북특별자치도 부안군 주산면   존재     2
49763  5280031021   전북특별자치도 부안군 주산면 소주리   존재     3
49764  5280031022   전북특별자치도 부안군 주산면 덕림리   존재     3
49765  5280031023   전북특별자치도 부안군 주산면 동정리   존재     3
...           ...                   ...  ...   ...
49856  5280042024    전북특별자치도 부안군 위도면 대리   존재     3
49857  5280042025   전북특별자치도 부안군 위도면 거륜리   존재     3
49858  5280042026   전북특별자치도 부안군 위도면 식도리   존재     3
49859  5280042027  전북특별자치도 부안군 위도면 상왕등리   존재     3
49860  5280042028  전북특별자치도 부안군 위도면 하왕등리   존재     3

[100 rows x 4 columns]


In [72]:
존재_지역['레벨'] = 존재_지역['공백개수']+1
존재_지역

,법정동코드,법정동명,폐지여부,공백개수,레벨
0,1100000000,서울특별시,존재,0,1
1,1111000000,서울특별시 종로구,존재,1,2
2,1111010100,서울특별시 종로구 청운동,존재,2,3
3,1111010200,서울특별시 종로구 신교동,존재,2,3
4,1111010300,서울특별시 종로구 궁정동,존재,2,3
...,...,...,...,...,...
49856,5280042024,전북특별자치도 부안군 위도면 대리,존재,3,4
49857,5280042025,전북특별자치도 부안군 위도면 거륜리,존재,3,4
49858,5280042026,전북특별자치도 부안군 위도면 식도리,존재,3,4
49859,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재,3,4


In [18]:
insert_data = """
insert into regions (region_code, reigon_name, parent_code, region_level)
values (%s, %s, %s, %s)
"""

_IncompleteInputError: incomplete input (1142305772.py, line 1)

In [76]:
# 법정동코드를 문자열로 변환 (앞자리 0 유지)
존재_지역['법정동코드'] = 존재_지역['법정동코드'].astype(str).str.zfill(10)

# 레벨별 상위지역코드 생성
def get_parent_code(row):
    code = row['법정동코드']
    level = row['레벨']
    
    if level == 1:  # 시도
        return '0000000000'
    elif level == 2:  # 시군구 → 시도코드 (앞 2자리)
        return code[:2] + '00000000'
    elif level == 3:  # 읍면동 → 시군구코드 (앞 5자리)
        return code[:5] + '00000'
    elif level == 4:  # 리 → 읍면동코드 (앞 8자리)
        return code[:8] + '00'

존재_지역['상위지역코드'] = 존재_지역.apply(get_parent_code, axis=1)

# 확인
print(존재_지역[['법정동코드', '법정동명', '레벨', '상위지역코드']].tail(20))

            법정동코드                  법정동명  레벨      상위지역코드
49841  5280040023   전북특별자치도 부안군 하서면 석상리   4  5280040000
49842  5280040024   전북특별자치도 부안군 하서면 백련리   4  5280040000
49843  5280040025   전북특별자치도 부안군 하서면 장신리   4  5280040000
49844  5280041000       전북특별자치도 부안군 줄포면   3  5280000000
49845  5280041021   전북특별자치도 부안군 줄포면 줄포리   4  5280041000
49846  5280041022   전북특별자치도 부안군 줄포면 장동리   4  5280041000
49847  5280041023   전북특별자치도 부안군 줄포면 우포리   4  5280041000
49848  5280041024    전북특별자치도 부안군 줄포면 신리   4  5280041000
49849  5280041025   전북특별자치도 부안군 줄포면 난산리   4  5280041000
49850  5280041026   전북특별자치도 부안군 줄포면 파산리   4  5280041000
49851  5280041027   전북특별자치도 부안군 줄포면 대동리   4  5280041000
49852  5280042000       전북특별자치도 부안군 위도면   3  5280000000
49853  5280042021    전북특별자치도 부안군 위도면 진리   4  5280042000
49854  5280042022   전북특별자치도 부안군 위도면 정금리   4  5280042000
49855  5280042023   전북특별자치도 부안군 위도면 치도리   4  5280042000
49856  5280042024    전북특별자치도 부안군 위도면 대리   4  5280042000
49857  5280042025   전북특별자치도 부안군 위도면 거륜리   4  528

In [79]:
존재_지역

,법정동코드,법정동명,폐지여부,공백개수,레벨,상위지역코드
0,1100000000,서울특별시,존재,0,1,0000000000
1,1111000000,서울특별시 종로구,존재,1,2,1100000000
2,1111010100,서울특별시 종로구 청운동,존재,2,3,1111000000
3,1111010200,서울특별시 종로구 신교동,존재,2,3,1111000000
4,1111010300,서울특별시 종로구 궁정동,존재,2,3,1111000000
...,...,...,...,...,...,...
49856,5280042024,전북특별자치도 부안군 위도면 대리,존재,3,4,5280042000
49857,5280042025,전북특별자치도 부안군 위도면 거륜리,존재,3,4,5280042000
49858,5280042026,전북특별자치도 부안군 위도면 식도리,존재,3,4,5280042000
49859,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재,3,4,5280042000


In [80]:
with MySQLConnector() as db:
    # DataFrame을 튜플 리스트로 변환
    data_tuples = [
        (
            row['법정동코드'],
            row['법정동명'],
            row['상위지역코드'],
            row['레벨']
        )
        for _, row in 존재_지역.iterrows()
    ]
    
    # 한 번에 삽입
    db.cursor.executemany("""
        insert into regions (region_code, region_name, parent_code, region_level)
        values (%s, %s, %s, %s)
    """, data_tuples)
    
    db.conn.commit()
    print(f"✅ {len(data_tuples)}건 삽입 완료")

✅ MySQL 연결 성공
✅ 20555건 삽입 완료
✅ MySQL 연결 종료
